In [3]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [8]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [9]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [10]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [16]:
from sklearn.model_selection import cross_validate
score = cross_validate(dt, train_input, train_target)
print(score)

{'fit_time': array([0.01082587, 0.0076921 , 0.00763607, 0.00795221, 0.00718832]), 'score_time': array([0.00144434, 0.00073528, 0.00076556, 0.00079536, 0.00071406]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [20]:
import numpy as np
print(np.mean(score['test_score']))

0.855300214703487


In [21]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [23]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [24]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)
gs.fit(train_input, train_target)

dt=gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [25]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [26]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [28]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [30]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth':range(5, 20, 1),
          'min_samples_split':range(2, 100, 10)}

In [31]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [34]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [36]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8615384615384616


In [38]:
from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([9, 6, 7, 3, 9, 3, 2, 7, 1, 1])

In [39]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101, 103,  98,  99,  90,  99,  94, 106, 106, 104]))

In [40]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.37868319, 0.54003968, 0.48177515, 0.18882078, 0.54081993,
       0.55183803, 0.05955125, 0.39000031, 0.51707397, 0.63773964])

In [41]:
params = {'min_impurity_decrease':uniform(0.0001,0.001),
          'max_depth':randint(20, 50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)}

In [43]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f931c3219d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f931c321750>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f931c05b810>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f931c05b450>},
                   random_state=42)

In [45]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [49]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [50]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
